In [26]:
"""
Author: Pengbo Li
Created: 2024-10-22
Version: 1.0

Description: 
    Simulation of the Corecoil default design created by Fred, inspired from the I2PS LVDT design.
    Check the Voice coil actuator performance of air coil + magnet core

Model:  
- air coil + magnet core
"""
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from modules import simulator, geometry, coreConfig, dataHandler

def run_vc_simulation(path, filename, simulation_freq, simulation_amplitude, core_params,  coil_params):
    """
    Run the LVDT simulation with specified parameters and save the results to a file.

    Parameters:
        path (str): Path to save the data file.
        filename (str): Name of the data file.
        simulation_freq (float): Simulation frequency in Hz.
        simulation_amplitude (float): Amplitude for the outer coil circuit.
        core_params (tuple): Tuple containing parameters for the magnet core.
        coil_params (dict): Dictionary containing parameters for CoreCoil, MiddleCoil, and OuterCoil.
    """
    # Define geometry for core and coils
    Magnet_geo = geometry.def_core_geo(*core_params['MagnetCore'])
    CoreCoil_geo = geometry.def_coil_geo(*coil_params['CoreCoil'])
    MiddleCoil_geo = geometry.def_coil_geo(*coil_params['MiddleCoil'])
    OuterCoil_geo = geometry.def_coil_geo(*coil_params['OuterCoil'])

    # Define circuit properties
    CC_circuit = simulator.def_circuit_prop("corecoil", 0, 0)
    MC_circuit = simulator.def_circuit_prop("middlecoil", 0, 0)
    OC_upper_circuit = simulator.def_circuit_prop("outercoil_upper", simulation_freq, simulation_amplitude)
    OC_lower_circuit = simulator.def_circuit_prop("outercoil_lower", simulation_freq, -simulation_amplitude)

    # Set up simulation environment
    simulator.def_femm_problem(signal_frequency=simulation_freq)
    simulator.build_air_geometry("Outside", 10)

    # Build geometry and assign labels
    m_label = simulator.build_core_geometry(Magnet_geo, 1)
    cc_label = simulator.build_coil_geometry(CoreCoil_geo, CC_circuit, 2, customized_material=True)
    mc_label = simulator.build_coil_geometry(MiddleCoil_geo, MC_circuit, 3, customized_material=True)
    oc_upper_label = simulator.build_coil_geometry(OuterCoil_geo, OC_upper_circuit, 4, customized_material=True)
    oc_lower_label = simulator.build_coil_geometry(OuterCoil_geo, OC_lower_circuit, 5, customized_material=True, reverse=True)

    print("Magnet label:", m_label)
    print("Core coil label:", cc_label)
    print("Middle coil label:", mc_label)
    print("Outer coil upper label:", oc_upper_label)
    print("Outer coil lower label:", oc_lower_label)
    
    config = coreConfig.moving_config(-5, 1, 10)
    vc_data = simulator.def_vc_data(config['steps'])
    sim_results = simulator.vc_simulation(
        moving_parts_label=[1, 2],
        CC_config=config,
        vc_data=vc_data,
        M_label=m_label,
        MC_label=mc_label,
        OC_upper_label=oc_upper_label,
        OC_lower_label=oc_lower_label,
    )
    # Save results
    dataHandler.save_data(sim_results, path + filename)
    print("Data saved to:", path + filename)

In [ ]:
# Example usage
path = "../data/aircoil/"

simulation_freq = 0
simulation_amplitude = 1

core_params = {
    'MagnetCore': (8, 4, "N40")
}

coil_params = {
    'CoreCoil': ("100um", 0.1, 0, 8, 12, 12, 0),
    'MiddleCoil': ("100um", 0.1, 0, 16, 3, 18, 0),
    'OuterCoil': ("100um", 0.1, 0, 16, 3, 18, 16)
}
filename_template = 'VC_aircoil_magnet_d{diam:02d}_l{length:02d}.h5'
magnet_diam = [6,8]
magnet_length = [4]
magnet_iter = coreConfig.params_iter(magnet_diam, magnet_length)

for iter in magnet_iter:
    core_params['MagnetCore'] = (iter[0], iter[1], "N40")
    filename = filename_template.format(diam=iter[0], length=iter[1])
    run_vc_simulation(path, filename, simulation_freq, simulation_amplitude, core_params, coil_params)

